In [1]:
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
import dill
import gzip
import logging

sys.path.append(os.path.abspath('../../surmise/emulationmethods'))
from AKSGP import Emulator

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        # logging.FileHandler('emulator_train.log', mode='w'),  # Log to file
        logging.StreamHandler()  # Log to console
    ]
)
logger = logging.getLogger(__name__)

# Try mock data

In [ ]:
from sklearn.datasets import make_friedman1, make_friedman2
X, y = make_friedman2(n_samples=50, noise=0.0, random_state=0)
emu = Emulator(X=X, Y_mean=y, Y_std=None)
emu.fit(kernel='AKS', nrestarts=5)

Xtest, ytest = make_friedman2(n_samples=10, noise=0.0, random_state=0)
means, std_devs = emu.predict(Xtest)
print("% error:\n", (1.0 - means/ytest.reshape(-1, 1))*100)

# Load heavy-ion data

In [2]:
# Load training data
train_dir = 'simulation_data/Grad_Pb-Pb-2760GeV/train'
X = np.loadtxt(os.path.join(train_dir, 'X.txt'))
Ymean = np.loadtxt(os.path.join(train_dir, 'Ymean.txt'))
Ystd = np.loadtxt(os.path.join(train_dir, 'Ystd.txt'))

print(f"Arrays loaded from directory '{train_dir}'.")
print(f"Shapes of loaded arrays: {X.shape}, {Ymean.shape}, {Ystd.shape}")


# Load testing data
test_dir = 'simulation_data/Grad_Pb-Pb-2760GeV/test'

Xval = np.loadtxt(os.path.join(test_dir, 'X.txt'))
Ymeanval = np.loadtxt(os.path.join(test_dir, 'Ymean.txt'))
Ystdval = np.loadtxt(os.path.join(test_dir, 'Ystd.txt'))

print(f"Arrays loaded from directory '{test_dir}'.")
print(f"Shapes of loaded arrays: {Xval.shape}, {Ymeanval.shape}, {Ystdval.shape}")


Arrays loaded from directory 'simulation_data/Grad_Pb-Pb-2760GeV/train'.
Shapes of loaded arrays: (485, 17), (485, 110), (485, 110)
Arrays loaded from directory 'simulation_data/Grad_Pb-Pb-2760GeV/test'.
Shapes of loaded arrays: (93, 17), (93, 110), (93, 110)


# Train emulators

In [4]:
import time
import psutil

# Start wall-clock time for reference (optional)
start_wall_time = time.time()

# Record the start CPU times
start_cpu_times = psutil.cpu_times()

inpsize = 485; numobs= (5, 23, 33, 40, 51, 57, 67, 75, 85, 95, 100, 106);

emu = Emulator(X=X[:inpsize], Y_mean=Ymean[:inpsize, numobs], Y_std=Ystd[:inpsize, numobs])
emu.fit(kernel='AKS', nrestarts=5, seed=13)


# Record the end CPU times
end_cpu_times = psutil.cpu_times()

# End wall-clock time (optional)
end_wall_time = time.time()

# Calculate the total CPU time
user_time = end_cpu_times.user - start_cpu_times.user
system_time = end_cpu_times.system - start_cpu_times.system
total_cpu_time = user_time + system_time

# Calculate the wall-clock time
wall_time_elapsed = end_wall_time - start_wall_time

# Log the results
print(f"Total CPU time taken: {total_cpu_time:.2f} seconds")
print(f"Total wall-clock time taken: {wall_time_elapsed:.2f} seconds (optional)")


# emu = Emulator(X=X, Y_mean=Ymean, Y_std=Ystd)
# emu.fit(kernel='AKS', nrestarts=5)

INFO:__main__:start time: 1724885018.5824609
INFO:AKSGP:Automatic kernel selection opted. Best kernel for each output dimension will be selected from the list of kernels:
   ['Matern12', 'Matern32', 'Matern52', 'RBF']

INFO:AKSGP:Shape of training arrays: (436, 17), (436, 12), (436, 12)
INFO:AKSGP:Shape of validation arrays: (49, 17), (49, 12), (49, 12)
INFO:AKSGP:Training GPs with all available kernels...
INFO:AKSGP:  Standardizing input space...
INFO:AKSGP:  Standardizing data...
/Users/jaiswal/miniconda3/envs/jssims_new/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:455: ConvergenceWarning: The optimal value found for dimension 4 of parameter k2__length_scale is close to the specified upper bound 1000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/jaiswal/miniconda3/envs/jssims_new/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:455: ConvergenceWarning: The optimal value found for dimension 15 of para

In [5]:
print(f"Total CPU time taken: {total_cpu_time:.2f} seconds")

Total CPU time taken: 923.28 seconds


In [7]:
(1724885167.367305-1724885018.5824609)*8

1190.2787532806396

In [ ]:
import pandas as pd
test = tuple(np.random.randint(0, 93, 10).flatten())

Ymean_test =  Ymean[test, :]
Ymean_test =  Ymean_test[:, numobs]
Ystd_test =  Ystd[test, :]
Ystd_test =  Ystd_test[:, numobs]

Ymeanval_test =  Ymeanval[test, :]
Ymeanval_test =  Ymeanval_test[:, numobs]
Ystdval_test =  Ystdval[test, :]
Ystdval_test =  Ystdval_test[:, numobs]

pd.set_option('display.float_format', '{:,.7f}'.format)
print("Predictions on training set ----------------->")
# Predict with standard deviation
means, std_devs = emu.predict(X[test, :])

df_means_error = (1- means/Ymean_test)*100
df_std_ratio = std_devs/Ystd_test
df_means_error_rounded = pd.DataFrame(df_means_error).round(6)
df_std_ratio_rounded = pd.DataFrame(df_std_ratio).round(6)

print("  % error in means:")
print(df_means_error_rounded.to_string(index=False, header=False))
print("\n  ratio of std:")
print(df_std_ratio_rounded.to_string(index=False, header=False))


# print("  Original means:\n", Ymeanval_test,"\n")
# print("  Predicted means:\n", means,"\n")

# print("  Original standard deviations:\n", Ystdval_test,"\n")
# print("  Predicted standard deviations:\n", std_devs,"\n")


print("\nPredictions on test set ----------------->")
# Predict with standard deviation
means, std_devs = emu.predict(Xval[test, :])

df_means_error = (1- means/Ymeanval_test)*100
df_std_ratio = (std_devs/Ystdval_test)
df_means_error_rounded = pd.DataFrame(df_means_error).round(6)
df_std_ratio_rounded = pd.DataFrame(df_std_ratio).round(6)


print("  % error in means:")
print(df_means_error_rounded.to_string(index=False, header=False))
print("\n  ratio of std:")
print(df_std_ratio_rounded.to_string(index=False, header=False))

# print("  Original means:\n", Ymeanval_test,"\n")
# print("  Predicted means:\n", means,"\n")

# print("  Original standard deviations:\n", Ystdval_test,"\n")
# print("  Predicted standard deviations:\n", std_devs,"\n")


In [ ]:
min_values = np.min(X, axis=0)
max_values = np.max(X, axis=0)
X_test = np.array([np.linspace(min_val, max_val, 100) for min_val, max_val in zip(min_values, max_values)]).T

y_pred, y_std = emu.predict(X_test)

In [ ]:
numobs= (5, 23, 33, 40, 51, 57, 67, 75, 85, 95, 100, 106)
# Set up the subplots
fig, axes = plt.subplots(nrows=6, ncols=3, figsize=(15, 20))
axes = axes.flatten()

# Loop through each feature
for param_no in range(X.shape[1]):
    obs_no = 5  # Adjust this if you want to visualize different observations
    
    axes[param_no].plot(X_test[:, param_no], y_pred[:, obs_no], 'b-', label='GP Mean')
    # pp = obs_no+1
    # axes[param_no].plot(X_test[:, param_no+1], y_pred[:, pp], 'g-')
    # axes[param_no].plot(X_test[:, param_no], y_pred[:, pp], 'g-')
    axes[param_no].fill_between(X_test[:, param_no], 
                                y_pred[:, obs_no] - 1.96 * y_std[:, obs_no], 
                                y_pred[:, obs_no] + 1.96 * y_std[:, obs_no], 
                                alpha=0.2, color='blue', label='95% Confidence Interval')
    axes[param_no].set_xlabel(f'Feature {param_no + 1}')
    axes[param_no].set_ylabel('Target')
    axes[param_no].set_title(f'GP Regression Along Feature {param_no + 1}')
    axes[param_no].legend()

# Remove any empty subplots if fewer than 18 features
for i in range(X.shape[1], len(axes)):
    fig.delaxes(axes[i])

plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Set up the observation range
numobs = (5, 23, 33, 40, 51, 57, 67, 75, 85, 95, 100, 106)
obs_range = range(len(numobs))  # This will generate obs_no from 0 to 9
n_features = X.shape[1]  # Number of features

# Loop through each obs_no
for obs_no in obs_range:
    # Set up the subplots for the current obs_no
    fig, axes = plt.subplots(nrows=6, ncols=3, figsize=(15, 20))
    axes = axes.flatten()
    # print("Observable: ", numobs[obs_no], "\n")
    # Loop through each feature
    for param_no in range(n_features):
        axes[param_no].plot(X_test[:, param_no], y_pred[:, obs_no], 'b-', label=f'GP Mean Obs {obs_no}')
        
        # Fill confidence interval for the current feature
        axes[param_no].fill_between(X_test[:, param_no], 
                                    y_pred[:, obs_no] - 1.96 * y_std[:, obs_no], 
                                    y_pred[:, obs_no] + 1.96 * y_std[:, obs_no], 
                                    alpha=0.2, color='blue', label='95% Confidence Interval')
        
        axes[param_no].set_xlabel(f'Feature {param_no + 1}')
        axes[param_no].set_ylabel(f'Observable: {numobs[obs_no]}')
        axes[param_no].set_title(f'GP Regression Obs {numobs[obs_no]} Feature {param_no + 1}')
        axes[param_no].legend()

    # Remove any empty subplots if fewer than 18 features
    for i in range(n_features, len(axes)):
        fig.delaxes(axes[i])

    plt.tight_layout()
    plt.show()
